# Bi-Directional Attention Flow Model

#### Set Checkpoint and Data Folder

In [12]:
!pip install -U PyYAML
!pip install -U h5py
!pip install pytorch-lightning
!pip install matplotlib

Requirement already up-to-date: PyYAML in /home/julian/Development/PythonEnv/pytorch/lib/python3.8/site-packages (5.3.1)
Requirement already up-to-date: h5py in /home/julian/Development/PythonEnv/pytorch/lib/python3.8/site-packages (2.10.0)


In [2]:
import sys, os
import torch
pwd = os.getcwd()

class Arguments():
    data = os.path.join(pwd, 'DATA', 'train_v2.1.json')
    exp_folder = os.path.join(pwd, 'Experimente/LightningTest')
    word_rep = os.path.join(pwd, 'DATA', 'glove.840B.300d.txt')
    #word_rep = None
    cuda = torch.cuda.is_available()
    use_covariance = False
    force_restart = False
    train_original_data = os.path.join(pwd, 'DATA', 'train_v2.1.json')
    train_splitted_data = os.path.join(pwd, 'DATA', 'train_part.json')
    val_data = os.path.join(pwd, 'DATA', 'dev_v2.1.json')
    test_splitted_data = os.path.join(pwd, 'DATA', 'eval_part.json')
    val_splitted_data = os.path.join(pwd, 'DATA', 'dev_part.json')

args = Arguments()

if not os.path.exists(args.exp_folder):
    os.makedirs(args.exp_folder)


#### Global Configurations (instead of config.yaml)

In [3]:
import yaml

config_yaml = """
    bidaf:
        dropout: 0.2
        num_highways: 2
        num_lstm: 2
        hidden_size: 100
        embedding_dim: 300
        embedding_reduce: 100
        characters:
            dim: 16
            num_filters: 100
            filter_sizes:
                - 5
    training:
        lr: 0.001
        betas:
            - 0.9
            - 0.999
        eps: 0.00000001
        weigth_decay: 0
        epochs: 1
        batch_size: 60
        limit: 400
"""
config = yaml.load(config_yaml, Loader=yaml.FullLoader)


#### Importing the MSMARCO Bidaf Model

In [4]:
sys.path.append(os.path.join(pwd,'MsmarcoQuestionAnswering','Baseline'))
sys.path.append(os.path.join(pwd,'MsmarcoQuestionAnswering','Baseline','scripts'))

import MsmarcoQuestionAnswering.Baseline.scripts.checkpointing as checkpointing
import MsmarcoQuestionAnswering.Baseline.scripts.train as train_manager
#import MsmarcoQuestionAnswering.Baseline.scripts.predict as predict_manager
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
import h5py

In [5]:
import MsmarcoQuestionAnswering.Baseline.mrcqa as mrcqa
import MsmarcoQuestionAnswering.Baseline.scripts.dataset as dataset
import json as json
import numpy as np
from random import shuffle

def try_to_split_testset(percentual_size_test, reduced_whole_size=1, force_renew=False):
    if os.path.isfile(args.train_splitted_data) and os.path.isfile(args.test_splitted_data) and os.path.isfile(args.val_splitted_data) and not force_renew:
        return;
    else:
        args.force_restart = True
        with open(args.train_original_data) as f_o:
            train_json = json.load(f_o);
        qids = list(train_json['query_id'].keys());
        shuffle(qids);
        train_size = len(qids)
        train_size = int(reduced_whole_size * train_size)
        new_train_size = int((1 - percentual_size_test) * train_size)
        new_test_size = train_size - new_train_size
        print("New Train Set has {} Datapoints".format(new_train_size))
        print("New Test Set has {} Datapoints".format(new_test_size))

        
        qids_train = qids[0:new_train_size]
        qids_test = qids[new_train_size:train_size]
        
        def copy_dict_part(old_dict, qids):
            count = 0;
            new_dict = dict()
            keys = old_dict.keys()
            for qid in qids:
                count = count + 1;
                if count % 10000 == 0:
                    print('Copy progress: {}'.format(count/len(qids)))
                for key in keys:
                    if not key in new_dict:
                        new_dict[key] = dict()
                    new_dict[key][qid] = train_json[key][qid]
            return new_dict;
        
        print('Start creating new train set:')
        new_train = copy_dict_part(train_json, qids_train)
        print('Start creating new test set:')
        new_test = copy_dict_part(train_json, qids_test)
        
        with open(args.train_splitted_data, 'w') as write_f:
            write_f.write(json.dumps(new_train))
        with open(args.test_splitted_data, 'w') as write_f:
            write_f.write(json.dumps(new_test))
            
        with open(args.val_data) as f_o:
            val_json = json.load(f_o);
        qids = list(val_json['query_id'].keys())
        shuffle(qids)
        val_size = len(qids)
        new_val_size = int(reduced_whole_size * val_size)
        print("New Validation Set has {} Datapoints".format(new_val_size))

        qids_val = qids[0:new_val_size]
        print('Start creating new val set:')
        new_val = copy_dict_part(val_json, qids_val)
        with open(args.val_splitted_data, 'w') as write_f:
            write_f.write(json.dumps(new_val))
            
def load_data(path,limit):
    with open(path) as f_o:
        data, _ = dataset.load_data(json.load(f_o), span_only=True, answered_only=True, loading_limit=limit)
    return data

def init_model(id_to_token, id_to_char):
    return mrcqa.BidafModel.from_config(config['bidaf'], id_to_token, id_to_char)

def reload_model(checkpoint):
    model, id_to_token, id_to_char = mrcqa.BidafModel.from_checkpoint(config['bidaf'], checkpoint)
    if torch.cuda.is_available() and args.cuda:
        model.cuda()
    model.train()
    return model, id_to_token, id_to_char

def inverse_dict(base_dict):
    return {tok: id_ for id_, tok in base_dict.items()}

def get_loader(data, config, used_data_per_batch=1.0):
    data = dataset.EpochGen(
        data,
        batch_size=config.get('training', {}).get('batch_size', 32),
        shuffle=True,
        used_data_per_batch=used_data_per_batch)
    return data

def get_optimizer(model, config, state):
    """
    Get the optimizer
    """
    parameters = filter(lambda p: p.requires_grad,
                        model.parameters())
    optimizer = torch.optim.Adam(
        parameters,
        lr=config['training'].get('lr', 0.01),
        betas=config['training'].get('betas', (0.9, 0.999)),
        eps=config['training'].get('eps', 1e-8),
        weight_decay=config['training'].get('weight_decay', 0))

    if state is not None:
        optimizer.load_state_dict(state)

    return optimizer

def load_pretrained_embeddings(path, model, id_to_token):
    with open(path) as f_o:
            pre_trained = dataset.SymbolEmbSourceText(f_o, set(tok for id_, tok in id_to_token.items() if id_ != 0))
    mean, cov = pre_trained.get_norm_stats(args.use_covariance)
    rng = np.random.RandomState(2)
    oovs = dataset.SymbolEmbSourceNorm(mean, cov, rng, args.use_covariance)
    model.embedder.embeddings[0].embeddings.weight.data = torch.from_numpy(dataset.symbol_injection(id_to_token, 0, model.embedder.embeddings[0].embeddings.weight.data.numpy(), pre_trained, oovs))

In [6]:
def new_init(train_path, val_path, test_path, config, args, loading_limit=None, used_data_per_train_batch=1.0):
    token_to_id = {'': 0}
    char_to_id = {'': 0}
    print('Load Train Data [1/6]')
    train_data = load_data(train_path,loading_limit)
    print('Load Validation Data [1/6]')
    val_data = load_data(val_path,loading_limit)
    print('Load Test Data [1/6]')
    test_data = load_data(test_path,loading_limit)
    
    print('Tokenize Train Data [2/6]')
    train_data = dataset.tokenize_data(train_data, token_to_id, char_to_id)
    print('Tokenize Validation Data [2/6]')
    val_data = dataset.tokenize_data(val_data, token_to_id, char_to_id)
    print('Tokenize Test Data [2/6]')
    test_data = dataset.tokenize_data(test_data, token_to_id, char_to_id)
    
    train_loader = get_loader(train_data, config, used_data_per_batch=used_data_per_train_batch)
    val_loader = get_loader(val_data, config)
    test_loader = get_loader(test_data, config)

    print('Create Inverse Dictionaries [3/6]')
    id_to_token = inverse_dict(token_to_id)
    id_to_char = inverse_dict(char_to_id)

    print('Initiate Model [4/6]')
    model = init_model(id_to_token, id_to_char)

    if args.word_rep:
        print('Load pre-trained embeddings [5/6]')
        load_pretrained_embeddings(args.word_rep, model, id_to_token)
    else:
        print('No pre-trained embeddings given [5/6]')
        pass  # No pretraining, just keep the random values.

    if torch.cuda.is_available() and args.cuda:
        model.cuda()
    model.train()

    optimizer = get_optimizer(model, config, state=None)
    print('Done init_state [6/6]')
    return model, id_to_token, id_to_char, optimizer, train_loader, val_loader, test_loader   


def new_reload(train_path, val_path, test_path, checkpoint, training_state, config, args,loading_limit=None, used_data_per_train_batch=1.0):
    print('Load Model from Checkpoint [1/5]')
    model, id_to_token, id_to_char = reload_model(checkpoint)

    optimizer = get_optimizer(model, config, training_state)

    print('Create Inverse Dictionaries [2/5]')
    token_to_id = inverse_dict(id_to_token)
    char_to_id = inverse_dict(id_to_char)

    len_tok_voc = len(token_to_id)
    len_char_voc = len(char_to_id)

    print('Load Train Data [3/5]')
    train_data = load_data(train_path,loading_limit)
    print('Load Validation Data [3/5]')
    val_data = load_data(val_path,loading_limit)
    print('Load Test Data [1/6]')
    test_data = load_data(test_path,loading_limit)
    
    limit_passage = config.get('training', {}).get('limit')

    print('Tokenize Train Data [4/5]')
    train_data = dataset.tokenize_data(train_data, token_to_id, char_to_id)
    print('Tokenize Validation Data [4/5]')
    val_data = dataset.tokenize_data(val_data, token_to_id, char_to_id)
    print('Tokenize Test Data [4/5]')
    test_data = dataset.tokenize_data(test_data, token_to_id, char_to_id)

    train_loader = get_loader(train_data, config, used_data_per_batch=used_data_per_train_batch)
    val_loader = get_loader(val_data, config)
    test_loader = get_loader(test_data, config)

    assert len(token_to_id) == len_tok_voc
    assert len(char_to_id) == len_char_voc

    print('Done reload_state [5/5]')
    return model, id_to_token, id_to_char, optimizer, train_loader, val_loader, test_loader


#### Pytorch Lightning Wrapper

In [7]:
try_to_split_testset(0.1,0.2, False);

checkpoint_w, training_state_w, epoch_w = train_manager.try_to_resume(
            args.force_restart, args.exp_folder)

if checkpoint_w:
    print('Resuming training...')
    model_w, id_to_token_w, id_to_char_w, optimizer_w, train_loader, val_loader, test_loader = new_reload(args.train_splitted_data, args.val_splitted_data, 
                                                                                                          args.test_splitted_data, checkpoint_w, 
                                                                                                          training_state_w, config, args, used_data_per_train_batch=0.3)
else:
    print('Preparing to train...')
    model_w, id_to_token_w, id_to_char_w, optimizer_w, train_loader, val_loader, test_loader = new_init(args.train_splitted_data, args.val_splitted_data, 
                                                                                                        args.test_splitted_data,config, args, used_data_per_train_batch=0.3)
    checkpoint_w = h5py.File(os.path.join(args.exp_folder, 'checkpoint'))
    checkpointing.save_vocab(checkpoint_w, 'vocab', id_to_token_w)
    checkpointing.save_vocab(checkpoint_w, 'c_vocab', id_to_char_w)

if torch.cuda.is_available() and args.cuda:
    train_loader.tensor_type = torch.cuda.LongTensor
    val_loader.tensor_type = torch.cuda.LongTensor
    #test_loader.tensor_type = torch.cuda.LongTensor

/home/julian/Development/QuestionAnweringProject/MSMARCO-Question-Answering/MsmarcoQuestionAnswering/Baseline/scripts/train.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  checkpoint = h5py.File(exp_folder + '/checkpoint')


Resuming training...
Load Model from Checkpoint [1/5]


/home/julian/Development/PythonEnv/pytorch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Create Inverse Dictionaries [2/5]
Load Train Data [3/5]
Start Organizing Data...
Organizing progress: 0.0 x 10⁴
Organizing progress: 1.0 x 10⁴
Organizing progress: 2.0 x 10⁴
Organizing progress: 3.0 x 10⁴
Organizing progress: 4.0 x 10⁴
Organizing progress: 5.0 x 10⁴
Organizing progress: 6.0 x 10⁴
Organizing progress: 7.0 x 10⁴
Organizing progress: 8.0 x 10⁴
Organizing progress: 9.0 x 10⁴
Organizing progress: 10.0 x 10⁴
Organizing progress: 11.0 x 10⁴
Organizing progress: 12.0 x 10⁴
Organizing progress: 13.0 x 10⁴
Organizing progress: 14.0 x 10⁴
Load Validation Data [3/5]
Start Organizing Data...
Organizing progress: 0.0 x 10⁴
Organizing progress: 1.0 x 10⁴
Organizing progress: 2.0 x 10⁴
Load Test Data [1/6]
Start Organizing Data...
Organizing progress: 0.0 x 10⁴
Organizing progress: 1.0 x 10⁴
Tokenize Train Data [4/5]
0.0 x 10⁴/6.3374 x 10⁴
1.0 x 10⁴/6.3374 x 10⁴
2.0 x 10⁴/6.3374 x 10⁴
3.0 x 10⁴/6.3374 x 10⁴
4.0 x 10⁴/6.3374 x 10⁴
5.0 x 10⁴/6.3374 x 10⁴
6.0 x 10⁴/6.3374 x 10⁴
Tokenize 

In [10]:
epoch_saves = dict();
epoch_saves['train_loss'] = []
epoch_saves['val_loss'] = []
epoch_saves['test_loss'] = []


class BidafLightningWrapper(LightningModule):
    def __init__(self):
        super().__init__()
        
    def setup(self,stage):
        pass;
            
    def prepare_data(self):
        pass;

    def configure_optimizers(self):
        return optimizer_w;

    def forward(self, passage, p_lengths, question, q_lengths):
        return model_w(passage, p_lengths, question, q_lengths)

    def train_dataloader(self):
        return train_loader;
    
    def val_dataloader(self):
        return val_loader;
    
    def test_dataloader(self):
        return test_loader;

    def training_step(self, batch, batch_idx):
        qids, passages, queries, answers, _ = batch
        start_log_probs, end_log_probs = self(passages[:2], passages[2], queries[:2], queries[2])
        loss = model_w.get_loss(start_log_probs, end_log_probs, answers[:, 0], answers[:, 1])
        return {'loss': loss, 'train_loss': loss, 'log': {'train_loss': loss}}

    def training_epoch_end(self, results):
        checkpointing.checkpoint(model_w, epoch_w, optimizer_w, checkpoint_w, args.exp_folder)
        model_w.cuda()
        mean_loss = self.save_statistics('train',results)
        return {'log': {'train_loss': mean_loss}}
    
    def validation_step(self, batch, batch_idx):
        qids, passages, queries, answers, _ = batch
        start_log_probs, end_log_probs = self(passages[:2], passages[2], queries[:2], queries[2])
        loss = model_w.get_loss(start_log_probs, end_log_probs, answers[:, 0], answers[:, 1])
        return {'val_loss': loss, 'log': {'val_loss': loss}}
    
    def validation_epoch_end(self, results):
        val_loss_mean = self.save_statistics('val',results)
        return {'val_loss': val_loss_mean}

    def test_step(self, batch, batch_idx):
        qids, passages, queries, answers, _ = batch
        start_log_probs, end_log_probs = self(passages[:2], passages[2], queries[:2], queries[2])
        loss = model_w.get_loss(start_log_probs, end_log_probs, answers[:, 0], answers[:, 1])
        return {'test_loss': loss}

    def test_epoch_end(self, results):
        mean_test_loss = self.save_statistics('test',results)
        return {'log': {'test_loss': mean_test_loss}}

    def save_statistics(self, phase, results):
        key = phase + '_loss'
        mean_loss = torch.stack([step[key] for step in results]).mean()
        print("Mean {} Loss: {}".format(phase,mean_loss))
        print(epoch_saves.keys())
        epoch_saves[key].append([step['val_loss'] for step in results])
        return mean_loss



In [11]:
modelLightning = BidafLightningWrapper()
trainer = Trainer(max_epochs=2, gpus=1)
trainer.fit(modelLightning)
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------


Validation sanity check: 100%|██████████| 2/2 [00:00<00:00,  4.62it/s]Mean val Loss: 2.7524495124816895
dict_keys(['train_loss', 'val_loss', 'test_loss'])
Epoch 1:   6%|▋         | 27/420 [00:08<02:10,  3.02it/s, loss=2.627, v_num=8]


/home/julian/Development/PythonEnv/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
from pathlib import Path
outputfile = os.path.join(args.exp_folder,'statistics.json')
Path(outputfile).touch()

#### Save Loss to statistics.json

In [ ]:
def writeStatisticToDict(key, output_dict):
    output_dict[key] = dict()
    stat_saves = epoch_saves[key]

    for idx in range(0,len(stat_saves)):
        output_dict[key][idx] = dict()
        for jdx in range(0,len(stat_saves[idx])):
            output_dict[key][idx][jdx] = stat_saves[idx][jdx].item() 

In [ ]:
output_dict = dict()

writeStatisticToDict('train_loss',output_dict)
writeStatisticToDict('val_loss',output_dict)
writeStatisticToDict('test_loss',output_dict)

with open(outputfile, 'w') as write_f:
    write_f.write(json.dumps(output_dict))

#### Load Loss from statistics.json

In [ ]:
with open(outputfile, 'r') as read_f:
    statistics = json.load(read_f)

In [ ]:
from matplotlib import pyplot as plt
def visualizeLoss(key, stats):
    batch_stats = stats[key]
    epoch_stats = []
    for key in batch_stats:
        epoch_stats = epoch_stats + list(batch_stats[key].values())
    plt.plot(epoch_stats)

In [ ]:
visualizeLoss('train_loss', statistics)
visualizeLoss('val_loss',statistics)
visualizeLoss('test_loss',statistics)